# Modelos de Ensemble
Actividad Lección 10 || Programación Python para Machine Learning

Objetivos:
* Presentar los principios en los que se basan los modelos de Ensemble.
* Conocer las distintas estrategias de Ensemble.
* Dominar las técnicas de implementación de los modelos de Ensemble en Python.
* Examinar los puntos clave que determinan el rendimiento de los modelos de Ensemble. 

Datos del alumno:
* Víctor Luque Martín
* Máster Avanzado en Programación en Python para Hacking, BigData y Machine Learning

Fecha: 02/01/2023

# Tabla de Contenidos
1. [Importes](#importes)
2. [Carga de datos](#carga)
    1. [Wine Dataset](#wine)
    2. [Computer Hardware Dataset](#hardware)
3. [Preparación de datos](#preparacion)
    1. [Wine Dataset](#wine_prep)
    2. [Computer Hardware Dataset](#hardware_prep)
4. [Modelo base](#modelo_base)
5. [Modelos de Ensemble](#ensemble)
    1. [Bagging](#bagging)
        1. [Bagging para regresión](#bagging_reg)
        2. [Bagging para clasificación](#bagging_clas)
    2. [Boosting](#boosting)
        1. [Boosting para regresión](#boosting_reg)
        2. [Boosting para clasificación](#boosting_clas)
6. [Evaluación de modelos](#evaluacion)
    1. [Evaluación de modelos de regresión](#evaluacion_reg)
    2. [Evaluación de modelos de clasificación](#evaluacion_clas)
7. [Conclusiones](#conclusiones)

# Importes <a name="importes"></a>

In [1]:
import random, time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    KFold
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    BaggingClassifier, 
    BaggingRegressor,
    AdaBoostClassifier,
    AdaBoostRegressor,
)

# Carga de datos <a class="anchor" name="carga"></a>
Para trabajar con los modelos de Ensemble, se utilizarán los siguientes datasets:
* [Wine](https://archive.ics.uci.edu/ml/datasets/wine): Para problemas de clasificación.
* [Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware): Para problemas de regresión.

## Wine Dataset <a class="anchor" name="wine"></a>
Estos datos son los resultados de un análisis químico de vinos cultivados en la misma región de Italia pero procedentes de tres cultivares diferentes. El análisis determinó las cantidades de 13 componentes presentes en cada uno de los tres tipos de vino.

In [2]:
wine_names = [
    'class','alcohol','malic_acid','ash',
    'alcalinity_of_ash','magnesium',
    'total_phenols','flavanoids',
    'nonflavanoid_phenols','proanthocyanins',
    'color_intensity','hue',
    'od280/od315_of_diluted_wines','proline'
]
df_wine_clf = pd.read_csv('wine.data', header=None, names=wine_names)
df_wine_clf.head()

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


## Computer Hardware Dataset <a class="anchor" name="hardware"></a>
Este conjunto de datos contiene información sobre el rendimiento de los microprocesadores.

In [3]:
hardware_names = [
    'vendor_name','model_name','myct','mmin','mmax',
    'cach','chmin','chmax','prp','erp'
]
df_hardware = pd.read_csv('machine.data', header=None, names=hardware_names)
df_hardware.head()

,vendor_name,model_name,myct,mmin,mmax,cach,chmin,chmax,prp,erp
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


# Preparación de datos <a class="anchor" name="preparacion"></a>
Antes de comenzar a trabajar con los modelos de Ensemble, se deben preparar los datos para que puedan ser utilizados por los modelos.

## Wine Dataset <a class="anchor" name="wine_prep"></a>
Se separan en variables independientes y dependientes.

In [4]:
seed = random.seed(time.time())
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(
    df_wine_clf.drop(columns=['class']), df_wine_clf['class'], test_size=0.3, random_state=seed
)

## Computer Hardware Dataset <a class="anchor" name="hardware_prep"></a>
De acuerdo con la información del dataset, se deben eliminar aquellas columnas que no aporten información al modelo. En este caso, se eliminarán las columnas:
* **Vendor Name**: Nombre del vendedor, un total de 30
* **Model Name**: Nombre del modelo con muchos simbolos únicos.
* **ERP**: Rendimiento relativo estimado

Se mantienen las columnas que si aportan información al modelo:
* **MYCT**: Tiempo de ciclo medio en nanosegundos (variable independiente)
* **MMIN**: Tamaño mínimo de memoria en kilobytes (variable independiente)
* **MMAX**: Tamaño máximo de memoria en kilobytes (variable independiente)
* **CACH**: Tamaño del caché en kilobytes (variable independiente)
* **CHMIN**: Número mínimo de canales (variable independiente)
* **CHMAX**: Número máximo de canales (variable independiente)
* **PRP**: Rendimento relativo publicado (`variable dependiente`)

In [5]:
df_hardware.drop(columns=['vendor_name','model_name', 'erp'], inplace=True)
df_hardware.head()

,myct,mmin,mmax,cach,chmin,chmax,prp
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


Se separan en variables independientes y dependientes.

In [6]:
seed = random.seed(time.time())
X_hardware_train, X_hardware_test, y_hardware_train, y_hardware_test = train_test_split(
    df_hardware.drop(columns=['prp']), df_hardware['prp'], test_size=0.3, random_state=seed
)

# Modelo base <a class="anchor" name="modelo_base"></a>
Se planteará un modelo base para cada tipo de problema (regresión y clasificación), posteriormente se utilizará en los modelos de Ensemble. Los modelos base serán:
* Regresión: DecisionTreeRegressor
* Clasificación: SVC

In [7]:
# Decision Tree Regressor
seed = random.seed(time.time())
base_reg = DecisionTreeRegressor(max_depth=5, random_state=seed, ccp_alpha=0.01)

In [8]:
# SVC
seed = random.seed(time.time())
base_clf = SVC(C=.1, kernel='sigmoid', gamma=.01, class_weight='balanced', max_iter=100, random_state=seed, probability=True)

# Modelos de Ensemble <a class="anchor" name="ensemble"></a>
Existen distintas estrategias de Ensemble, en este caso se utilizarán las siguientes:
* Bagging
* Boosting

## Bagging <a class="anchor" name="bagging"></a>
Los métodos de Bagging son un método de Ensemble que consiste en tomar múltiples instancias con reemplazo del conjunto de datos de entrenamiento y entrenar un modelo para cada muestra. A la hora de la predicción, para calcular la salida final del Ensemble se combinan las predicciones de todos
los modelos base.

Esta combinación de predicciones se realiza mediante un promedio para la regresión, o mediante votación para la clasificación, normalmente, por mayoría.

Cuando los resultados se combinan de este modo, la varianza general del modelo disminuye y presenta un mejor rendimiento como resultado

### Bagging para regresión <a class="anchor" name="bagging_reg"></a>
Implementando el modelo BaggingRegressor de Scikit-Learn.

In [9]:
seed = random.seed(time.time())
bagg_reg = BaggingRegressor(estimator=base_reg, n_estimators=100, random_state=seed, max_samples=0.3)

### Bagging para clasificación <a class="anchor" name="bagging_clas"></a>
Implementando el modelo BaggingClassifier de Scikit-Learn.

In [10]:
seed = random.seed(time.time())
bagg_clf = BaggingClassifier(estimator=base_clf, n_estimators=100, random_state=seed, max_samples=0.3)

## Boosting <a class="anchor" name="boosting"></a>
Los modelos de Boosting trabajan en todas las etapas con el conjunto completo de entrenamiento, y se manipulan los pesos de las instancias para generar modelos distintos

La idea se basa en que en cada iteración se incremente el peso de las instancias mal predichas por el modelo, provocando que en el entrenamiento del siguiente predictor estos objetos tendrán una mayor importancia y tendrán más probabilidades de ser correctamente predichas.

### Boosting para regresión <a class="anchor" name="boosting_reg"></a>
Implementando el modelo AdaBoostRegressor de Scikit-Learn.

In [11]:
seed = random.seed(time.time())
boost_reg = AdaBoostRegressor(estimator=base_reg, n_estimators=100, random_state=seed, learning_rate=0.3)

### Boosting para clasificación <a class="anchor" name="boosting_clas"></a>
Implementando el modelo AdaBoostClassifier de Scikit-Learn.

In [12]:
seed = random.seed(time.time())
boost_clf = AdaBoostClassifier(estimator=base_clf, n_estimators=100, random_state=seed, learning_rate=0.3)

# Evaluación de modelos <a class="anchor" name="evaluacion"></a>
Se evaluarán los modelos utilizando KFold Cross Validation. Dependiendo del tipo de problema se emplearán unas métricas u otras.

In [13]:
seed = random.seed(time.time())
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

## Evaluación de modelos de regresión <a class="anchor" name="evaluacion_reg"></a>
Se utilizarán las siguientes métricas:
* Mean Absolute Error
* Root Mean Squared Error
* R2 Score

In [14]:
metrics_reg = ['neg_mean_absolute_error', 'neg_root_mean_squared_error', 'r2']
cv_base = [cross_val_score(base_reg, X_hardware_train, y_hardware_train, cv=kfold, scoring=metric) for metric in metrics_reg]
cv_bagg = [cross_val_score(bagg_reg, X_hardware_train, y_hardware_train, cv=kfold, scoring=metric) for metric in metrics_reg]
cv_boost = [cross_val_score(boost_reg, X_hardware_train, y_hardware_train, cv=kfold, scoring=metric) for metric in metrics_reg]
rendimiento_reg = pd.DataFrame(data={
    'Base Model (mean)': [cv.mean() for cv in cv_base],
    'Base Model (std)': [cv.std() for cv in cv_base],
    'Bagging Model (mean)': [cv.mean() for cv in cv_bagg],
    'Bagging Model (std)': [cv.std() for cv in cv_bagg],
    'Boosting Model (mean)': [cv.mean() for cv in cv_boost],
    'Boosting Model (std)': [cv.std() for cv in cv_boost]
}, index=['MAE', 'RMSE', 'R2'])
rendimiento_reg.index.name = 'Métricas Regresión'
rendimiento_reg.loc["Average"] = rendimiento_reg.mean()
rendimiento_reg = rendimiento_reg.abs()
rendimiento_reg

,Base Model (mean),Base Model (std),Bagging Model (mean),Bagging Model (std),Boosting Model (mean),Boosting Model (std)
Métricas Regresión,,,,,,
MAE,42.094621,11.612642,35.562654,16.150240,39.459108,10.827707
RMSE,80.109072,48.681408,78.025997,43.295106,86.449240,38.253277
R2,0.292207,1.998564,0.798584,0.109712,0.647888,0.151019
Average,40.831966,20.764205,37.596689,19.851686,41.753486,16.410668


## Evaluación de modelos de clasificación <a class="anchor" name="evaluacion_clas"></a>
Se utilizarán las siguientes métricas:
* Accuracy
* Balanced Accuracy
* F1-Score
* AUC

In [15]:
metrics_clf = ['accuracy', 'balanced_accuracy', 'f1_weighted', 'roc_auc_ovr']
cv_base = [cross_val_score(base_clf, X_wine_train, y_wine_train, cv=kfold, scoring=metric) for metric in metrics_clf]
cv_bagg = [cross_val_score(bagg_clf, X_wine_train, y_wine_train, cv=kfold, scoring=metric) for metric in metrics_clf]
cv_boost = [cross_val_score(boost_clf, X_wine_train, y_wine_train, cv=kfold, scoring=metric) for metric in metrics_clf]
rendimiento_clf = pd.DataFrame(data={
    'Base Model (mean)': [cv.mean() for cv in cv_base],
    'Base Model (std)': [cv.std() for cv in cv_base],
    'Bagging Model (mean)': [cv.mean() for cv in cv_bagg],
    'Bagging Model (std)': [cv.std() for cv in cv_bagg],
    'Boosting Model (mean)': [cv.mean() for cv in cv_boost],
    'Boosting Model (std)': [cv.std() for cv in cv_boost]
}, index=['Accuracy', 'Balanced Accuracy', 'F1-Score', 'ROC AUC'])
rendimiento_clf.index.name = 'Métricas Clasificación'
rendimiento_clf.loc["Average"] = rendimiento_clf.mean()
rendimiento_clf

,Base Model (mean),Base Model (std),Bagging Model (mean),Bagging Model (std),Boosting Model (mean),Boosting Model (std)
Métricas Clasificación,,,,,,
Accuracy,0.330333,0.116163,0.347000,0.075073,0.329667,0.100401
Balanced Accuracy,0.333333,0.000000,0.333333,0.000000,0.333333,0.000000
F1-Score,0.134578,0.055157,0.227659,0.070969,0.201506,0.033347
ROC AUC,0.500000,0.000000,0.500000,0.000000,0.500000,0.000000
Average,0.324561,0.042830,0.351998,0.036510,0.341127,0.033437


# Conclusiones <a class="anchor" name="conclusiones"></a>
Tras observar las evaluaciones podemos concluir que la estrategia de Bagging es la que mejor rendimiento presenta en ambos tipos de problemas.

El Árbol de Decisión para el problema de regresión muestra un error bastante elevado, tras realizar el Ensemble con Bagging, el error disminuye ligéramente. Sería recomendable optimizar los hiperparámetros del modelo base para mejorar el rendimiento y volver a evaluar el rendimiento del modelo y del Ensemble.

El SVM para el problema de clasificación muestra un rendimiento bastante bajo, ya que el conjunto de datos es muy pequeño y no es posible realizar un buen entrenamiento. Tras realizar el Ensemble de cualquier estrategia, el rendimiento aumenta ligeramente, pero sigue siendo bajo. Sería recomendable utilizar otro tipo de modelo para este problema, o optimizar los hiperparámetros del modelo base para mejorar el rendimiento y volver a evaluar el rendimiento del modelo y del Ensemble.